In [1]:
import pandas as pd
import os

import scipy.stats
from scipy.stats import hypergeom
import statsmodels.stats.multitest as smm
import matplotlib.pyplot as plt
from itertools import combinations
%pylab inline
import collections
import numpy

Populating the interactive namespace from numpy and matplotlib


In [2]:
#these files were generated using '101319 cofactor analysis.ipynb' and '101319  single factor analysis.ipynb'

ba_df=pd.read_csv("../../output/072020_single_factor_analysis_data_DE_NB_ba.csv",sep=',')

sc_df=pd.read_csv("../../output/072020_single_factor_analysis_data_DE_NB_sc.csv",sep=',')

#this file is generated in 'Figure 1 data 101319.ipynb' -this file is independent of data source (ba or sc)
gdata=pd.read_csv("../../data/genetic_data/071420_genetic_data_TFdep_gt_TF_indep.csv",sep=',')

In [3]:
data_source='sc'
df=sc_df
df['data_source']='single cell'

In [9]:
data_source='ba'
df=ba_df
df["data_source"]='Brain Atlas'

In [22]:
df['prop_w_motif']=df['obs_motif']/df['marker_count']
run_these_TS={'DA':['unc-3'],'DB':['unc-3'],'VA':['unc-3'],'VB':['unc-3'],
           'ASE':['che-1'],'AIY':['ceh-10--ttx-3','ceh-10','ttx-3'],'ADL':['hlh-4']}
run_these_val={'PVC':['cfi-1'],'URX': ['unc-86'], 'SMB':['lim-4', 'unc-42'], 'SMD':['lim-4', 'unc-42'],'SMDD':['lim-4', 'unc-42'], 'AWA':['odr-7'] }

In [23]:
run_these_x={'DVA':['fax-1'],'ASI': ['unc-3'],'PVQ': ['pag-3']}
run_these_x={'BDU':['unc-86']}

In [24]:
run_these=run_these_x
cols=['data_source','model','neuron_class', 'TF', 'motif_ID','marker_count', 'obs_motif','prop_w_motif','fc','hg_pval*','rs_pval*','best_subset_rs_pval*']
d=pd.DataFrame()
for neuron_class in run_these.keys():
    #print(neuron_class,run_these[neuron_class])
    for TF in run_these[neuron_class]:
        r=df.loc[(df["neuron_class"]==neuron_class) & (df["TF"]==TF)][cols]
        d=d.append(r)
d

data_source model neuron_class      TF     motif_ID  marker_count  \
764  single cell   A-I          BDU  unc-86  M03312_2.00           259   
795  single cell   A-I          BDU  unc-86  M02244_2.00           259   

     obs_motif  prop_w_motif        fc      hg_pval*      rs_pval*  \
764        228      0.880309  1.276902  1.540000e-12  1.200000e-27   
795        141      0.544402  1.889327  2.990000e-17  6.960000e-09   

     best_subset_rs_pval*  
764          3.620000e-57  
795          3.790000e-23

In [46]:
#ceh-32 C-I reg. in IL1in sc data, (B-II reg in IL1 in BA data)
df.loc[df["model"].isin(['C-I','C-II'])]

Unnamed: 0 model      TF     motif_ID neuron_class  marker_count  \
988         988   C-I  ceh-32  M00483_2.00          IL1            40   

     obs_motif  exp_motif        fc   hg_pval   rs_pval  best_subset_rs_pval  \
988         11   6.385986  1.722522  0.044563  0.158857                  1.0   

     num_genes_in_subset  hg_pval*  rs_pval*  best_subset_rs_pval*  \
988                    0  0.049925  0.180802                   1.0   

     data_source  prop_w_motif  
988  single cell         0.275

In [10]:
df.loc[df['TF']=='unc-42'].sort_values(ascending=True,by=['model','rs_pval*'])

Unnamed: 0 model      TF     motif_ID neuron_class  marker_count  \
826         826   A-I  unc-42  M03874_2.00          AIB          54.0   
829         829   A-I  unc-42  M03874_2.00          AVB          57.0   
828         828   A-I  unc-42  M03874_2.00          AVA          86.0   
840         840   A-I  unc-42  M03874_2.00          SMD          68.0   
830         830   A-I  unc-42  M03874_2.00          AVD          66.0   
831         831   A-I  unc-42  M03874_2.00          AVE          64.0   
838         838   A-I  unc-42  M03874_2.00          SAA          47.0   
832         832   A-I  unc-42  M03874_2.00          AVH          35.0   
834         834   A-I  unc-42  M03874_2.00          RIV          54.0   
833         833   A-I  unc-42  M03874_2.00          AVK          47.0   
835         835   A-I  unc-42  M03874_2.00          RMD          71.0   
839         839   A-I  unc-42  M03874_2.00          SIB          48.0   
836         836   A-I  unc-42  M03874_2.00          RMF          37.0   
827         827   A-I  unc-42  M03874_2.00          ASH         179.0   
837         837   A-I  unc-42  M03874_2.00          RMH          36.0   

     obs_motif  exp_motif        fc       hg_pval       rs_pval  \
826       29.0  16.323082  1.776625  2.678465e-04  2.573341e-07   
829       35.0  17.229919  2.031350  1.076506e-06  1.799743e-05   
828       53.0  25.996019  2.038774  1.628550e-09  2.884271e-05   
840       38.0  20.554992  1.848699  9.402217e-06  5.180206e-05   
830       36.0  19.950433  1.804472  3.273664e-05  1.318934e-04   
831       37.0  19.345874  1.912552  4.182315e-06  1.644974e-04   
838       29.0  14.207127  2.041229  7.825191e-06  2.519020e-04   
832       20.0  10.579775  1.890399  8.380400e-04  3.101595e-04   
834       32.0  16.323082  1.960414  9.064331e-06  4.578518e-04   
833       25.0  14.207127  1.759680  8.412473e-04  4.663175e-04   
835       38.0  21.461829  1.770585  3.522207e-05  8.914747e-04   
839       27.0  14.509406  1.860862  1.558250e-04  4.157292e-03   
836       23.0  11.184334  2.056448  5.774854e-05  5.884600e-03   
827       81.0  54.107992  1.497006  1.489750e-05  8.045023e-03   
837       21.0  10.882054  1.929783  4.259537e-04  1.622749e-02   

     best_subset_rs_pval  num_genes_in_subset      hg_pval*  rs_pval*  \
826         7.485437e-11                 20.0  5.875745e-04  0.000004   
829         1.488471e-10                 19.0  6.791382e-06  0.000100   
828         1.490629e-12                 26.0  5.359019e-08  0.000148   
840         7.536361e-11                 20.0  3.697943e-05  0.000238   
830         1.353300e-10                 21.0  1.029284e-04  0.000488   
831         3.338034e-10                 20.0  1.935389e-05  0.000581   
838         1.859611e-08                 16.0  3.191184e-05  0.000825   
832         4.081479e-06                 12.0  1.504790e-03  0.000973   
834         7.157199e-09                 17.0  3.625732e-05  0.001334   
833         2.061888e-07                 14.0  1.509108e-03  0.001355   
835         1.088052e-09                 19.0  1.083887e-04  0.002331   
839         8.221692e-08                 14.0  3.760604e-04  0.008082   
836         9.526468e-07                 11.0  1.644958e-04  0.010846   
827         1.228088e-14                 34.0  5.324581e-05  0.014004   
837         8.547786e-06                  9.0  8.592200e-04  0.025256   

     best_subset_rs_pval*  data_source  
826          2.521975e-10  Brain Atlas  
829          4.679476e-10  Brain Atlas  
828          8.052822e-12  Brain Atlas  
840          2.534590e-10  Brain Atlas  
830          4.304913e-10  Brain Atlas  
831          9.595572e-10  Brain Atlas  
838          3.629739e-08  Brain Atlas  
832          5.259206e-06  Brain Atlas  
834          1.537775e-08  Brain Atlas  
833          3.254701e-07  Brain Atlas  
835          2.783045e-09  Brain Atlas  
839          1.405162e-07  Brain Atlas  
836          1.339548e-06  Brain Atlas  
827          1.17796

In [25]:
df.loc[df['neuron_class']=='PVC'].sort_values(ascending=True,by=['model','rs_pval*'])

Unnamed: 0 model      TF      motif_ID neuron_class  marker_count  \
1698        1698   A-I  ztf-11   M00650_2.00          PVC           427   
4              4   A-I   cfi-1   M01667_2.00          PVC           427   
1053        1053   A-I   egl-5   M09581_2.00          PVC           427   
1204        1204   A-I   unc-3  OH2011_UNC-3          PVC           427   
863          863   A-I  ceh-14   M00435_2.00          PVC           427   
1168        1168   A-I   gei-3   M00711_2.00          PVC           427   
1609        1609   A-I   pag-3   M00619_2.00          PVC           427   
1276        1276   A-I  lin-31   M00665_2.00          PVC           427   
1113        1113   A-I  vab-15   M00555_2.00          PVC           427   
738          738   A-I  ceh-18   M00681_2.00          PVC           427   
1383        1383   A-I   ztf-3   M00640_2.00          PVC           427   
1289        1289   A-I  daf-16   M01989_2.00          PVC           427   
389          389   A-I  ceh-48   M01905_2.00          PVC           427   
623          623   A-I  ceh-58   M00676_2.00          PVC           427   
848          848   A-I   lim-7   M01242_2.00          PVC           427   
1505        1505   A-I   ztf-9   M00636_2.00          PVC           427   
506          506   A-I  ceh-74   M01248_2.00          PVC           427   
272          272   A-I  ceh-38   M03716_2.00          PVC           427   
946          946  A-II   nsy-7   M00966_2.00          PVC           427   

      obs_motif   exp_motif        fc       hg_pval       rs_pval  \
1698        311  274.317856  1.133721  8.350000e-05  4.070000e-36   
4           307  246.883946  1.243499  7.970000e-10  6.830000e-32   
1053        279  214.010003  1.303677  1.020000e-10  2.340000e-31   
1204        178   86.849258  2.049528  2.970000e-24  1.930000e-27   
863         317  245.290186  1.292347  2.250000e-13  9.730000e-27   
1168        234  145.117100  1.612491  4.280000e-19  1.200000e-25   
1609        278  210.588733  1.320109  2.190000e-11  9.380000e-25   
1276        312  253.386483  1.231321  1.520000e-09  5.740000e-21   
1113        237  156.847168  1.511025  1.270000e-15  4.040000e-20   
738         280  197.243655  1.419564  2.930000e-16  7.450000e-20   
1383        249  142.673335  1.745246  1.600000e-26  1.330000e-17   
1289        252  162.797203  1.547938  8.890000e-19  1.150000e-14   
389         277  207.316214  1.336123  4.920000e-12  1.050000e-13   
623         219  140.335822  1.560542  1.560000e-15  3.600000e-12   
848         249  158.440928  1.571564  2.220000e-19  7.110000e-12   
1505        205  113.433164  1.807232  8.530000e-22  8.500000e-12   
506         186   97.070568  1.916132  4.360000e-22  8.600000e-12   
272         275  208.782472  1.317160  5.010000e-11  7.260000e-10   
946         195  114.601921  1.701542  2.710000e-17  2.413604e-01   

      best_subset_rs_pval  num_genes_in_subset      hg_pval*      rs_pval*  \
1698         2.990000e-68                  202  1.255520e-04  2.000000e-34   
4            1.320000e-69                  189  2.310000e-09  2.350000e-30   
1053         1.050000e-62                  174  3.310000e-10  7.790000e-30   
1204         7.840000e-48                  128  5.020000e-23  4.550000e-26   
863          3.850000e-65                  179  1.020000e-12  2.150000e-25   
1168         1.880000e-51                  153  3.870000e-18  2.330000e-24   
1609         8.950000e-59                  158  7.730000e-11  1.650000e-23   
1276         7.200000e-58                  171  4.270000e-09  6.430000e-20   
1113         5.120000e-50                  142  7.730000e-15  4.200000e-19   
738          2.850000e-51                  145  1.930000e-15  7.620000e-19   
1383         1.420000e-47                  139  3.630000e-25  1.080000e-16   
1289         1.930000e-44                  137  7.810000e-18  7.260000e-14   
389          4.610000e-48                  148  1.910000e-11  5.830000e-13   
623          5.360000e-45       

In [44]:
df.loc[df['neuron_class']=='IL1'].sort_values(ascending=True,by=['model','rs_pval*'])

Unnamed: 0 model      TF     motif_ID neuron_class  marker_count  \
1693        1693   A-I  ztf-11  M00650_2.00          IL1          55.0   
737          737   A-I  ceh-18  M00681_2.00          IL1          55.0   
1376        1376   A-I   ztf-3  M00640_2.00          IL1          55.0   
253          253   A-I  ceh-38  M03716_2.00          IL1          55.0   
489          489   A-I  ceh-74  M01248_2.00          IL1          55.0   
1040        1040  A-II  ceh-43  M00423_2.00          IL1          55.0   
607          607  A-II  ceh-58  M00676_2.00          IL1          55.0   
1576        1576  A-II   ref-2  M00621_2.00          IL1          55.0   
371          371  A-II  ceh-48  M01905_2.00          IL1          55.0   
1602        1602  A-II  egl-43  M00611_2.00          IL1          55.0   
1499        1499  A-II   ztf-9  M00636_2.00          IL1          55.0   
998          998  B-II  ceh-32  M00483_2.00          IL1          55.0   

      obs_motif  exp_motif        fc       hg_pval   rs_pval  \
1693       44.0  35.333682  1.245271  8.595853e-03  0.000105   
737        34.0  25.406091  1.338262  1.420143e-02  0.001872   
1376       30.0  18.377128  1.632464  9.964926e-04  0.004381   
253        35.0  26.892356  1.301485  1.961138e-02  0.010291   
489        20.0  12.503235  1.599586  1.525416e-02  0.019008   
1040       33.0  19.986563  1.651109  2.984197e-04  0.043912   
607        36.0  18.076043  1.991586  7.403924e-07  0.051108   
1576       21.0  11.558923  1.816778  2.672646e-03  0.055629   
371        37.0  26.703494  1.385586  3.867481e-03  0.062825   
1602       23.0  12.623669  1.821974  1.407854e-03  0.066332   
1499       24.0  14.610829  1.642617  4.571017e-03  0.474994   
998        14.0   8.780731  1.594400  4.696800e-02  0.952245   

      best_subset_rs_pval  num_genes_in_subset  hg_pval*  rs_pval*  \
1693         4.779563e-09                 26.0  0.010792  0.000404   
737          8.843042e-09                 21.0  0.016792  0.004195   
1376         8.570536e-08                 18.0  0.001719  0.008422   
253          1.830318e-07                 16.0  0.022592  0.017182   
489          5.203931e-06                 10.0  0.017934  0.029077   
1040         1.690795e-08                 16.0  0.000642  0.060084   
607          1.075662e-08                 15.0  0.000005  0.068630   
1576         7.479035e-05                 13.0  0.003938  0.073858   
371          4.571171e-06                 16.0  0.005398  0.082192   
1602         1.614602e-05                 12.0  0.002296  0.086360   
1499         2.730908e-05                 10.0  0.006216  0.509115   
998          4.874973e-03                  5.0  0.050342  0.956825   

      best_subset_rs_pval*  data_source  
1693          1.060871e-08  Brain Atlas  
737           1.865872e-08  Brain Atlas  
1376          1.456836e-07  Brain Atlas  
253           2.908704e-07  Brain Atlas  
489           6.539699e-06  Brain Atlas  
1040          3.321519e-08  Brain Atlas  
607           2.217373e-08  Brain Atlas  
1576          8.294858e-05  Brain Atlas  
371           5.830259e-06  Brain Atlas  
1602          1.912699e-05  Brain Atlas  
1499          3.157508e-05  Brain Atlas  
998           4.959388e-03  Brain Atlas

In [27]:
run_these=run_these_TS
cols=['data_source','model','neuron_class', 'TF', 'motif_ID','marker_count', 'obs_motif','prop_w_motif','fc','hg_pval*','rs_pval*','best_subset_rs_pval*']
d=pd.DataFrame()
for neuron_class in run_these.keys():
    #print(neuron_class,run_these[neuron_class])
    for TF in run_these[neuron_class]:
        r=df.loc[(df["neuron_class"]==neuron_class) & (df["TF"]==TF)][cols]
        d=d.append(r)
d

data_source model neuron_class             TF             motif_ID  \
1198  single cell   A-I           DA          unc-3         OH2011_UNC-3   
1199  single cell   A-I           DB          unc-3         OH2011_UNC-3   
1209  single cell   A-I           VA          unc-3         OH2011_UNC-3   
1210  single cell   A-I           VB          unc-3         OH2011_UNC-3   
1596  single cell   A-I          ASE          che-1         OH2004_CHE-1   
182   single cell   A-I          AIY  ceh-10--ttx-3  OH2007_CEH-10_TTX-3   
1116  single cell   A-I          AIY         ceh-10          M00287_2.00   
1107  single cell   A-I          AIY          ttx-3          M02085_2.00   
67    single cell   A-I          ADL          hlh-4         OH2018_HLH-4   

      marker_count  obs_motif  prop_w_motif        fc      hg_pval*  \
1198           901        437      0.485017  2.384616  1.740000e-80   
1199          1280        523      0.408594  2.008878  5.080000e-65   
1209           366        231      0.631148  3.103078  2.300000e-69   
1210          1275        413      0.323922  1.592581  1.890000e-24   
1596           336        220      0.654762  1.638454  1.990000e-20   
182            177        131      0.740113  1.430396  2.650000e-09   
1116           177        104      0.587571  1.534925  5.770000e-08   
1107           177         86      0.485876  1.955375  2.500000e-11   
67             595        405      0.680672  1.665947  4.490000e-40   

          rs_pval*  best_subset_rs_pval*  
1198  2.540000e-48         2.060000e-105  
1199  6.400000e-48         2.180000e-116  
1209  2.050000e-35          5.440000e-63  
1210  2.960000e-25          2.370000e-84  
1596  1.070000e-26          6.780000e-58  
182   1.490000e-21          2.600000e-38  
1116  2.980000e-05          3.520000e-20  
1107  7.750153e-03          1.480000e-15  
67    2.180000e-21          2.960000e-65

In [28]:
run_these=run_these_TS
cols=['data_source','model','neuron_class', 'TF', 'motif_ID','marker_count', 'obs_motif','prop_w_motif','fc','hg_pval*','rs_pval*','best_subset_rs_pval*']
d=pd.DataFrame()
for neuron_class in run_these.keys():
    #print(neuron_class,run_these[neuron_class])
    for TF in run_these[neuron_class]:
        r=df.loc[(df["neuron_class"]==neuron_class) & (df["TF"]==TF)][cols]
        d=d.append(r)
d

data_source model neuron_class             TF             motif_ID  \
1198  single cell   A-I           DA          unc-3         OH2011_UNC-3   
1199  single cell   A-I           DB          unc-3         OH2011_UNC-3   
1209  single cell   A-I           VA          unc-3         OH2011_UNC-3   
1210  single cell   A-I           VB          unc-3         OH2011_UNC-3   
1596  single cell   A-I          ASE          che-1         OH2004_CHE-1   
182   single cell   A-I          AIY  ceh-10--ttx-3  OH2007_CEH-10_TTX-3   
1116  single cell   A-I          AIY         ceh-10          M00287_2.00   
1107  single cell   A-I          AIY          ttx-3          M02085_2.00   
67    single cell   A-I          ADL          hlh-4         OH2018_HLH-4   

      marker_count  obs_motif  prop_w_motif        fc      hg_pval*  \
1198           901        437      0.485017  2.384616  1.740000e-80   
1199          1280        523      0.408594  2.008878  5.080000e-65   
1209           366        231      0.631148  3.103078  2.300000e-69   
1210          1275        413      0.323922  1.592581  1.890000e-24   
1596           336        220      0.654762  1.638454  1.990000e-20   
182            177        131      0.740113  1.430396  2.650000e-09   
1116           177        104      0.587571  1.534925  5.770000e-08   
1107           177         86      0.485876  1.955375  2.500000e-11   
67             595        405      0.680672  1.665947  4.490000e-40   

          rs_pval*  best_subset_rs_pval*  
1198  2.540000e-48         2.060000e-105  
1199  6.400000e-48         2.180000e-116  
1209  2.050000e-35          5.440000e-63  
1210  2.960000e-25          2.370000e-84  
1596  1.070000e-26          6.780000e-58  
182   1.490000e-21          2.600000e-38  
1116  2.980000e-05          3.520000e-20  
1107  7.750153e-03          1.480000e-15  
67    2.180000e-21          2.960000e-65

In [29]:
run_these=run_these_val
cols=['data_source','model','neuron_class', 'TF', 'motif_ID','marker_count', 'obs_motif','prop_w_motif','num_genes_in_subset','fc','hg_pval*','rs_pval*','best_subset_rs_pval*']
d=pd.DataFrame()
for neuron_class in run_these.keys():
    #print(neuron_class,run_these[neuron_class])
    for TF in run_these[neuron_class]:
        r=df.loc[(df["neuron_class"]==neuron_class) & (df["TF"]==TF)][cols]
        d=d.append(r)
d

data_source model neuron_class      TF     motif_ID  marker_count  \
4     single cell   A-I          PVC   cfi-1  M01667_2.00           427   
785   single cell   A-I          URX  unc-86  M03312_2.00           353   
816   single cell   A-I          URX  unc-86  M02244_2.00           353   
1042  single cell   A-I          SMB   lim-4  M02182_2.00           534   
1043  single cell   A-I          SMD   lim-4  M02182_2.00           402   
832   single cell   A-I          SMD  unc-42  M03874_2.00           402   
1851  single cell   A-I          AWA   odr-7  M00697_2.00           386   

      obs_motif  prop_w_motif  num_genes_in_subset        fc      hg_pval*  \
4           307      0.718970                  189  1.243499  2.310000e-09   
785         284      0.804533                  154  1.166987  1.230000e-06   
816         155      0.439093                   84  1.523859  2.410000e-09   
1042        253      0.473783                  132  1.349616  6.190000e-09   
1043        243      0.604478                  133  1.721913  2.820000e-24   
832         218      0.542289                   98  1.793998  7.470000e-23   
1851        245      0.634715                  112  1.270442  1.040000e-07   

          rs_pval*  best_subset_rs_pval*  
4     2.350000e-30          2.050000e-68  
785   2.030000e-21          5.350000e-59  
816   3.680000e-09          4.980000e-29  
1042  6.640000e-14          1.120000e-47  
1043  7.290000e-14          2.270000e-47  
832   3.520000e-07          1.440000e-35  
1851  2.040000e-08          9.080000e-40

In [52]:
df.loc[(df["neuron_class"]=="ASI") ].sort_values(ascending=True, by=['model','rs_pval*'])

Unnamed: 0 model      TF      motif_ID neuron_class  marker_count  \
140          140   A-I   fos-1   M01144_2.00          ASI           372   
153          153   A-I   skn-1   M00596_2.00          ASI           372   
1326        1326   A-I   ztf-3   M00640_2.00          ASI           372   
24            24   A-I   mdl-1   M08183_2.00          ASI           372   
1581        1581   A-I  egl-43   M00611_2.00          ASI           372   
1108        1108  A-II   ttx-3   M02085_2.00          ASI           372   
1192        1192  A-II   unc-3  OH2011_UNC-3          ASI           372   
449          449  A-II  ceh-74   M01248_2.00          ASI           372   
566          566  A-II  ceh-58   M00676_2.00          ASI           372   
1020        1020  A-II  ceh-57   M00675_2.00          ASI           372   
1260        1260  A-II   fkh-8   M03026_2.00          ASI           372   
1152        1152  A-II  ceh-36   M00673_2.00          ASI           372   
1448        1448  A-II   ztf-9   M00636_2.00          ASI           372   
161          161  A-II   atf-5   M00593_2.00          ASI           372   
1641        1641   B-I  ztf-11   M00650_2.00          ASI           372   
1793        1793   B-I   nhr-6   M01298_2.00          ASI           372   
1861        1861   B-I  ztf-16   M00644_2.00          ASI           372   
1155        1155  B-II  ceh-90   M00677_2.00          ASI           372   
1744        1744  B-II   dmd-6   M00655_2.00          ASI           372   
215          215  B-II  ceh-38   M03716_2.00          ASI           372   
332          332  B-II  ceh-48   M01905_2.00          ASI           372   

      obs_motif   exp_motif        fc       hg_pval   rs_pval  \
140         143   96.397133  1.483447  6.010000e-08  0.001194   
153         153  120.945357  1.265034  2.647310e-04  0.004108   
1326        150  124.296208  1.206795  2.876007e-03  0.007069   
24          128  100.895790  1.268636  1.085603e-03  0.026393   
1581        112   85.381905  1.311753  7.852650e-04  0.032614   
1108        138   92.435354  1.492935  8.250000e-08  0.148329   
1192        103   75.662586  1.361307  3.736940e-04  0.163418   
449         101   84.567334  1.194315  2.498018e-02  0.166211   
566         152  122.259779  1.243254  6.685580e-04  0.199834   
1020        120   90.824724  1.321226  3.398570e-04  0.332305   
1260        148  122.630039  1.206882  3.132594e-03  0.332178   
1152        120   86.474171  1.387698  3.870000e-05  0.518187   
1448        126   98.822335  1.275015  9.951200e-04  0.691774   
161         146  113.355031  1.287989  1.712570e-04  0.791304   
1641        247  238.984174  1.033541  2.063957e-01  0.000013   
1793        176  159.841147  1.101093  4.927227e-02  0.008279   
1861        120  113.318005  1.058967  2.399391e-01  0.027007   
1155        181  167.264855  1.082116  8.210037e-02  0.096358   
1744        276  266.124216  1.037110  1.380636e-01  0.135313   
215         161  181.890117  0.885150  9.875475e-01  0.733769   
332         169  180.612720  0.935704  8.977468e-01  0.910075   

      best_subset_rs_pval  num_genes_in_subset      hg_pval*  rs_pval*  \
140          9.490000e-21                   63  1.340000e-07  0.001822   
153          3.390000e-22                   63  3.731490e-04  0.005834   
1326         1.790000e-21                   64  3.635522e-03  0.009685   
24           3.650000e-19                   53  1.430568e-03  0.033522   
1581         1.310000e-14                   51  1.051917e-03  0.040674   
1108         1.590000e-18                   53  1.800000e-07  0.169649   
1192         3.350000e-12                   38  5.169530e-04  0.185653   
449          1.400000e-13                   38  2.881852e-02  0.188710   
566          5.980000e-18                   55  9.027220e-04  0.225484   
1020         1.910000e-14                   45  4.708440e-04  0.360789   
1260         1.300000e-17                   55  3.946475e-03  0.360789   
1152         3.350000e-15           

In [40]:
df.loc[(df["neuron_class"]=="AVK") & (df["TF"]=="fax-1")]

Unnamed: 0 model     TF     motif_ID neuron_class  marker_count  \
1834        1834   A-I  fax-1  M06432_2.00          AVK           104   

      obs_motif  exp_motif        fc   hg_pval   rs_pval  best_subset_rs_pval  \
1834         79  64.990943  1.215554  0.002411  0.000077         5.110000e-17   

      num_genes_in_subset  hg_pval*  rs_pval*  best_subset_rs_pval*  \
1834                   40  0.003073  0.000137          7.170000e-17   

      data_source  prop_w_motif  
1834  single cell      0.759615

In [51]:
df.loc[(df["neuron_class"]=="RIS") & (df["TF"]=="lim-6")]

Unnamed: 0 model     TF     motif_ID neuron_class  marker_count  \
1084        1084   A-I  lim-6  M00432_2.00          RIS           173   

      obs_motif  exp_motif        fc  hg_pval   rs_pval  best_subset_rs_pval  \
1084         68   45.37225  1.498713  0.00011  0.007582         2.580000e-11   

      num_genes_in_subset  hg_pval*  rs_pval*  best_subset_rs_pval*  \
1084                   34  0.000162  0.010358          2.950000e-11   

      data_source  
1084  single cell

In [ ]:
## top predicted regualtor of URX?
cols=['model','TF','motif_ID','neuron_class']
print(df.loc[(df["neuron_class"]=="URX")].sort_values(ascending=True, by=['model','rs_pval*'])[cols].head(2))
#top predicted regualtor of PVC?
print(df.loc[(df["neuron_class"]=="PVC")].sort_values(ascending=True, by=['model','rs_pval*'])[cols].head(2))

#top predicted regulators of AWA
print(df.loc[(df["neuron_class"]=="AWA")].sort_values(ascending=True, by=['model','rs_pval*'])[cols].head(2))

#"" of SMB
print(df.loc[(df["neuron_class"]=="SMB")].sort_values(ascending=True, by=['model','rs_pval*'])[cols].head(2))

#"" of SMD
print(df.loc[(df["neuron_class"]=="SMD")].sort_values(ascending=True, by=['model','rs_pval*'])[cols].head(2))
